In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread("Image Input/Shape.jpg")

In [3]:
#fungsi untuk stack gambar
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0],list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(0,rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[x][y].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y],(0,0),None,scale,scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] =cv2.cvtColor(imgArray[x][y],cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height,width,3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x],(0,0),None,scale,scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x],(imgArray[0].shape[0],imgArray[0].shape[1]),None,scale,scale)
            if len(imgArray[x].shape) == 2: imgArray[x] =cv2.cvtColor(imgArray[x],cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver

In [78]:
def getContours(img):
    countours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in countours:
        area = cv2.contourArea(cnt)
        if area > 500:
            #cv2.drawContours(imgBlank, cnt, -1, (255,0,0),3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.04*peri, True)
            print(len(approx))
            
            x, y, w, h = cv2.boundingRect(approx)
            
            cv2.rectangle(imgBlank, (x-10,y-10), (x+w+10, y+h+10), (0,255,0), 1)
            if len(approx) == 3 :
                objectType = "Triangle"
            elif len(approx) == 4:
                aspectRatio = w/float(h)
                print(aspectRatio)
                if aspectRatio > 0.95 and aspectRatio <1.5 :
                    objectType = "Square"
                else:
                    objectType = "Rectangle"
            else:
                objectType = "Circle"
                
            cv2.putText(imgBlank, objectType, (x , y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 2)

In [81]:
#Kernel sistem nya adalah mengganti value pixel dengan nilai pixel yang berada didekatnya
#Contoh pada tahap ini adalah 7X7, maka pada umumnya melakukan sum perhitungan dari 7x7 pixel di sekitarnya dengan target
#Pixel yang diganti adalah dipaling tengah(tidak bisa genap)
#https://setosa.io/ev/image-kernels/
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray,(7,7), 1)
imgCanny = cv2.Canny(imgBlur, 50,50)
imgBlank = img.copy()
(thresh, imgBlackWhite) = cv2.threshold(imgGray, 220, 255, cv2.THRESH_BINARY)

getContours(imgCanny)

imgStack = stackImages(1, ([img, imgBlur,imgCanny],
                          [imgGray,imgBlank, imgBlackWhite]))

4
1.896551724137931
6
4
1.162162162162162
4
1.896551724137931
8
4
1.1944444444444444
3
6
4
1.8620689655172413
4
1.162162162162162
4
1.8333333333333333
7
3
4
1.131578947368421
4
1.9642857142857142
7
3
4
1.131578947368421


In [82]:
cv2.imshow("output", imgStack)
cv2.waitKey(0)

-1